# CPA on Firmware Implementation of Kuznyechik (GOST)

## GOST Trace Capture

In [ ]:
SCOPETYPE = 'CWNANO' # options: OPENADC, CWNANO  
PLATFORM = 'CWNANO' # options: CWLITEXMEGA/CW308_XMEGA, CWLITEARM/CW308_STM32F3, CWNANO 
CRYPTO_TARGET='KUZNYECHIK'
SS_VER='SS_VER_1_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 50000 #options: 96000 for CW-PRO (CW1200), 24400 for CW-Lite, 131070 for CW-Husky

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../firmware/mcu/simpleserial-gost
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
cw.program_target(scope, prog, "../../firmware/mcu/simpleserial-gost/simpleserial-gost-{}.hex".format(PLATFORM))

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time
from os import urandom

trace_array = []
textin_array = []

text = urandom(16)

N = 350

for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    text = urandom(16)
    
trace_array = np.array(trace_array)

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert len(trace_array) == N
print("✔️ OK to continue!")

Again, let's quickly plot a trace to make sure everything looks as expected:

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.figure()
plt.plot(trace_array[0], 'r')
plt.plot(trace_array[N//2], 'g')
plt.plot(trace_array[N-1], 'b')
plt.show()

## Kuznyechik Model and Hamming Weight

In [ ]:
sbox = [
    252, 238, 221, 17, 207, 110, 49, 22, 251, 196, 250, 218, 35, 197, 4, 77, 233,
    119, 240, 219, 147, 46, 153, 186, 23, 54, 241, 187, 20, 205, 95, 193, 249, 24, 101,
    90, 226, 92, 239, 33, 129, 28, 60, 66, 139, 1, 142, 79, 5, 132, 2, 174, 227, 106, 143,
    160, 6, 11, 237, 152, 127, 212, 211, 31, 235, 52, 44, 81, 234, 200, 72, 171, 242, 42,
    104, 162, 253, 58, 206, 204, 181, 112, 14, 86, 8, 12, 118, 18, 191, 114, 19, 71, 156,
    183, 93, 135, 21, 161, 150, 41, 16, 123, 154, 199, 243, 145, 120, 111, 157, 158, 178,
    177, 50, 117, 25, 61, 255, 53, 138, 126, 109, 84, 198, 128, 195, 189, 13, 87, 223,
    245, 36, 169, 62, 168, 67, 201, 215, 121, 214, 246, 124, 34, 185, 3, 224, 15, 236,
    222, 122, 148, 176, 188, 220, 232, 40, 80, 78, 51, 10, 74, 167, 151, 96, 115, 30, 0,
    98, 68, 26, 184, 56, 130, 100, 159, 38, 65, 173, 69, 70, 146, 39, 94, 85, 47, 140, 163,
    165, 125, 105, 213, 149, 59, 7, 88, 179, 64, 134, 172, 29, 247, 48, 55, 107, 228, 136,
    217, 231, 137, 225, 27, 131, 73, 76, 63, 248, 254, 141, 83, 170, 144, 202, 216, 133,
    97, 32, 113, 103, 164, 45, 43, 9, 91, 203, 155, 37, 208, 190, 229, 108, 82, 89, 166,
    116, 210, 230, 244, 180, 192, 209, 102, 175, 194, 57, 75, 99, 182
]

kuz_lvec = [0x94, 0x20, 0x85, 0x10, 0xC2, 0xC0, 0x01, 0xFB,
            0x01, 0xC0, 0xC2, 0x10, 0x85, 0x20, 0x94, 0x01]

def kuz_mul_gf256(x, y):
    z = 0
    for i in range(8):
        if (y & 1):
            z ^= x
        if (x & 0x80):
            x = ((x << 1) & 0xFF) ^ 0xC3
        else:
            x = (x << 1) & 0xFF
        y >>= 1
    return z

def L(inputdata):
    res = [block for block in inputdata]
    x = 0
    for j in range(16):
        # An LFSR with 16 elements from GF(2^8)
        x = res[15] # since lvec[15] = 1

        for i in range(14,-1,-1):
            res[i+1] = res[i]
            x ^= kuz_mul_gf256(res[i], kuz_lvec[i])
        res[0] = x
    return res

def SX(inputdata, key):
    return sbox[inputdata ^ key]

HW = [bin(n).count("1") for n in range(0, 256)]

## Differential Attack Implementaiton

In [ ]:
# TODO: Implement the differential attack on the Kuznyechik cipher
# (use "SOLN_Lab 3_3 - DPA on Firmware Implementation of AES" from ../sca101 as a reference)

With one final check to make sure you've got the correct key:

In [ ]:
key = [0x6c,0xec,0xc6,0x7f,0x28,0x7d,0x08,0x3d,
       0xeb,0x87,0x66,0xf0,0x73,0x8b,0x36,0xcf,
       0x16,0x4e,0xd9,0xb2,0x46,0x95,0x10,0x90,
       0x86,0x9d,0x08,0x28,0x5d,0x2e,0x19,0x3b]

for bnum in range(32):
    assert bestguess[bnum] == key[bnum], \
    "Byte {} failed, expected {:02X} got {:02X}".format(bnum, key[bnum], bestguess[bnum])
print("✔️ OK to continue!")